In [8]:
from time import time
import json
import numpy as np
import os
from stable_baselines3 import TD3 as sb3TD3
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from IPython.display import clear_output
import matplotlib.pyplot as plt
import copy
from hyperopt import fmin, tpe, hp
import pickle
from torch import Tensor
# simple stopwatch to time whatevs, in (float) seconds
# keeps track of laps along with final time
class Stopwatch:
    def __init__(self):
            self.start_time = time()
            self.last_time = self.start_time
            self.laps = []
    def lap(self):
        this_time = time()
        delta_time = this_time - self.last_time
        self.laps.append(delta_time)
        self.last_time = this_time
        return delta_time
    def stop(self):
        self.stop_time = time()
        self.delta_time = self.stop_time - self.start_time
        return self.delta_time

In [2]:
nLayers = 3
nNodes = 2**5
learning_rate = 10**(int(-1*3))#params['learning_rate']))
weight_decay = 10**(int(-1*6))
eps = 1e-8#10**(int(-1*int(params['eps'])))
patience = 10#int(params['patience'])

activation_fn = nn.ReLU
with_bias = True
input_dim = 208
net_arch = [nNodes for _ in range(nLayers)]
output_dim = 2

modules = []
if nLayers == 0:
    modules.append(nn.Linear(input_dim, output_dim, bias=with_bias))
else:
    modules.append(nn.Linear(input_dim, net_arch[0], bias=with_bias))
    modules.append(activation_fn())
    for idx in range(len(net_arch) - 1):
        modules.append(nn.Linear(net_arch[idx], net_arch[idx + 1], bias=with_bias))
        modules.append(activation_fn())
    modules.append(nn.Linear(net_arch[-1], output_dim, bias=with_bias))
modules.append(nn.Tanh())
actor = nn.Sequential(*modules)
device = torch.device("cuda")
actor.cuda()
actor_weights = torch.load('actor_pretrained_weights_horz.pt')
print(actor)
actor.load_state_dict(actor_weights)

Sequential(
  (0): Linear(in_features=208, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=2, bias=True)
  (7): Tanh()
)


<All keys matched successfully>

In [6]:
nLayers = 3
nNodes = 2**5
learning_rate = 10**(int(-1*3))#params['learning_rate']))
weight_decay = 10**(int(-1*6))
eps = 1e-8#10**(int(-1*int(params['eps'])))
patience = 10#int(params['patience'])

activation_fn = nn.ReLU
with_bias = True
input_dim = 210
net_arch = [nNodes for _ in range(nLayers)]
output_dim = 1

modules = []
if nLayers == 0:
    modules.append(nn.Linear(input_dim, output_dim, bias=with_bias))
else:
    modules.append(nn.Linear(input_dim, net_arch[0], bias=with_bias))
    modules.append(activation_fn())
    for idx in range(len(net_arch) - 1):
        modules.append(nn.Linear(net_arch[idx], net_arch[idx + 1], bias=with_bias))
        modules.append(activation_fn())
    modules.append(nn.Linear(net_arch[-1], output_dim, bias=with_bias))
critic = nn.Sequential(*modules)
device = torch.device("cuda")
critic.cuda()
critic_weights = torch.load('critic_pretrained_weights_horz.pt')
print(critic)
critic.load_state_dict(critic_weights)

Sequential(
  (0): Linear(in_features=210, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): ReLU()
  (6): Linear(in_features=32, out_features=1, bias=True)
)


<All keys matched successfully>

In [7]:
sb3model = sb3TD3.load('model_in_horz.zip')
device = torch.device("cuda")
sb3model.actor.cuda()
sb3model.critic.cuda()
print(sb3model.actor)
print(sb3model.critic)

Actor(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mu): Sequential(
    (0): Linear(in_features=208, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=2, bias=True)
    (7): Tanh()
  )
)
ContinuousCritic(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (qf0): Sequential(
    (0): Linear(in_features=210, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
  (qf1): Sequential(
    (0): Linear(in_features=210, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3

In [8]:
sb3model.actor.mu = copy.deepcopy(actor)
sb3model.actor_target.mu = copy.deepcopy(actor)

In [9]:
for module in sb3model.critic.modules():
    del module
for module in sb3model.critic_target.modules():
    del module
for idx in range(len(sb3model.critic.q_networks)):
    q_net = copy.deepcopy(critic)
    sb3model.critic.q_networks[idx] = q_net
    sb3model.critic.add_module(f"qf{idx}", q_net)
    q_net_target = copy.deepcopy(critic)
    sb3model.critic_target.q_networks[idx] = q_net_target
    sb3model.critic_target.add_module(f"qf{idx}", q_net_target)

In [10]:
sb3model.save('model_out_horz.zip')